In [1]:
from codex import Codex

import logging

from codex.database.espresso import run_helpdoc, generate_database

logging.basicConfig(level=logging.DEBUG)

# Generate QE database

In [ ]:
work_dir = 'temp'
def_files = ['PW/Doc/INPUT_PW.def', 'PP/Doc/INPUT_PROJWFC.def']
versions = ['6.3']#, '6.8']#, '7.0', '7.2']
database_dir = 'database/'
gen_docs = True
from importlib import resources
database_dir = resources.files("codex.database")

if gen_docs:
    run_helpdoc(work_dir, def_files, versions, database_dir)
#generate_database(versions, database_dir)

# Generate Codex

In [2]:
from importlib import resources
filename = 'pw.in'
database_dir = resources.files("codex.database")
version = '7.2'
# Make .dft-tutor directory in current dir if it doesn't exist
Codex(filename, database_dir, version)


# CLI stuff

In [ ]:
from importlib.metadata import metadata
metadata = metadata('dft-codex')
__author__ = metadata['Author']
__version__ = metadata['Version']
__email__ = metadata['Author-email']
__maintainer__ = metadata['Maintainer']
__maintainer_email__ = metadata['Maintainer-email']
__summary__ = metadata['Summary']

print(list(metadata))
print(__author__)
print(__version__)
print(__email__)
print(__maintainer__)
print(__maintainer_email__)
print(__summary__)

from importlib.resources import files, contents
data_text = files('codex.database').joinpath('qe-7.2').joinpath('database.json')
# Check if it exists
data_text.exists()